In [ ]:
!git clone https://github.com/Olyco/EasyTSF.git

Cloning into 'EasyTSF'...
remote: Enumerating objects: 620, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 620 (delta 34), reused 37 (delta 20), pack-reused 560 (from 2)
Receiving objects: 100% (620/620), 1.80 MiB | 4.70 MiB/s, done.
Resolving deltas: 100% (336/336), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r "EasyTSF/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.

# ETTh1 датасет

## KAN

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/KAN/ETTh1_96for48.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
checkpoint directory created: drive/MyDrive/VKR/Results/save/110525_1424/model
saving model version 0.0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MultKAN | 334 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
282 K     Trainable params
52.1 K    Non-trainable params
334 K     Total params
1.336     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 134/134 [44:16<00:00, 19.83s/it, v_num=ed_1, train/loss_step=1.060]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation:   0% 0/45 [00:00<?, ?it/s]       
Validation DataLoader 0:   0% 0/45 [00:00<?, ?it/s]
Validation DataLoader 0:  44% 20/45 [01:23<01:43,  4.15s/it]
Valida

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/KAN/ETTh1_48for12.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
checkpoint directory created: drive/MyDrive/VKR/Results/save/110525_2132/model
saving model version 0.0
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MultKAN | 121 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
100 K     Trainable params
20.8 K    Non-trainable params
121 K     Total params
0.486     Total estimated model params size (MB)
9     

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/KAN/ETTh1_48for12_5.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
checkpoint directory created: drive/MyDrive/VKR/Results/save/120525_0129/model
saving model version 0.0
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MultKAN | 33.2 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
25.2 K    Trainable params
8.0 K     Non-trainable params
33.2 K    Total params
0.133     Total estimated model params size (MB)
9     

## Efficient KAN

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ETTh1_48for12.py # [336, 5, 84]

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 21.0 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
21.0 K    Trainable params
0         Non-trainable params
21.0 K    Total params
0.084     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 134/134 [00:01<00:00, 97.12it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ETTh1_48for12_20h_50ep.py # [336, 20, 20, 84]

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 88.0 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
88.0 K    Trainable params
0         Non-trainable params
88.0 K    Total params
0.352     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 134/134 [00:01<00:00, 73.52it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ETTh1_48for12_100h_100ep.py # [336, 100, 84]

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 420 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
420 K     Trainable params
0         Non-trainable params
420 K     Total params
1.680     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 134/134 [00:01<00:00, 95.71it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ETTh1_48for12_168h.py # [336, 168, 84]

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 705 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
705 K     Trainable params
0         Non-trainable params
705 K     Total params
2.822     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 134/134 [00:01<00:00, 90.78it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ETTh1_96for48_168h.py # [672, 168, 336]

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 1.7 M  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.774     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 132/132 [00:01<00:00, 94.90it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ETTh1_96for96_168h.py # [672, 168, 672]

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 2.3 M  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
2.3 M     Trainable params
0         Non-trainable params
2.3 M     Total params
9.032     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 132/132 [00:01<00:00, 91.13it/s, 

## MLP

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/ETTh1_96for48_100.py # [672, 100, 336]

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 101 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.405     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 132/132 [00:00<00:00, 140.47it/s,

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/ETTh1_48for12.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 42.2 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
42.2 K    Trainable params
0         Non-trainable params
42.2 K    Total params
0.169     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 134/134 [00:00<00:00, 152.14it/s,

# Полный датасет ETTh1 с нормализацией MinMax

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ETTh1_48for12_100h_100ep.py # [336, 10, 84]

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 42.0 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
42.0 K    Trainable params
0         Non-trainable params
42.0 K    Total params
0.168     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 134/134 [00:02<00:00, 54.27it/s, v_num=ed_1, train/loss_step=0.0142]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation:   0% 0/45 [00:00<?, ?it/s]       
Validation DataLoader 0:   0% 0/45 [00:00<?, ?it/s]
Validation DataLoader 0:   2% 1/45 [00:00<00:00, 55.65it/s]
Validation DataLoader 0:   4% 2/45 [00:00<00:00, 62.90it/s]
Validation DataLoader 0:   7% 3/45 [00:00<00:00, 70.87it/s]
Validation DataLoader 0:   9% 4/45 [0

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/ETTh1_48for12_5.py # [336, 10, 84]

Seed set to 1
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 4.3 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
4.3 K     Trainable params
0         Non-trainable params
4.3 K     Total params
0.017     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 134/134 [00:01<00:00, 109.30it/s, v_num=ed_1, train/loss_step=0.0161]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation:   0% 0/45 [00:00<?, ?it/s]       
Validation DataLoader 0:   0% 0/45 [00:00<?, ?it/s]
Validation DataLoader 0:   2% 1/45 [00:00<00:00, 188.75it/s]
Validation DataLoader 0:   4% 2/45 [00:00<00:00, 173.62it/s]
Validation DataLoader 0:   7% 3/45 [00:00<00:00, 200.57it/s]
Validation DataLoader 0:   9% 4/4

# ECL датасет

## efficient KAN

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ECL.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 481 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
481 K     Trainable params
0         Non-trainable params
481 K     Total params
1.926     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 328/328 [00:03<00:00, 83.40it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ECL.py #48for48

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 1.5 M  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
6.163     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 286/286 [00:05<00:00, 56.13it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ECL.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 481 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
481 K     Trainable params
0         Non-trainable params
481 K     Total params
1.926     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 287/287 [00:03<00:00, 84.16it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ECL.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 192 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
192 K     Trainable params
0         Non-trainable params
192 K     Total params
0.770     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 287/287 [00:03<00:00, 86.98it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/ECL.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 1.2 M  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.622     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 286/286 [00:04<00:00, 68.22it/s, 

## MLP

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/ECL.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 50.1 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
50.1 K    Trainable params
0         Non-trainable params
50.1 K    Total params
0.200     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 328/328 [00:02<00:00, 119.67it/s,

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/ECL.py #48for48

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 169 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
169 K     Trainable params
0         Non-trainable params
169 K     Total params
0.678     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 286/286 [00:03<00:00, 76.46it/s, 

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/ECL.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 50.1 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
50.1 K    Trainable params
0         Non-trainable params
50.1 K    Total params
0.200     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 287/287 [00:02<00:00, 123.10it/s,

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/ECL.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 21.2 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
21.2 K    Trainable params
0         Non-trainable params
21.2 K    Total params
0.085     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 287/287 [00:02<00:00, 119.81it/s,

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/ECL.py #48

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 130 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
130 K     Trainable params
0         Non-trainable params
130 K     Total params
0.524     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 286/286 [00:03<00:00, 86.39it/s, 

## N-BEATS

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/N_BEATS/ECL.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
prepare N-Beats data
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
   | Name                     | Type               | Params | Mode 
-------------------------------------------------------------------------
0  | loss                     | TorchMetricWrapper | 0      | train
1  | logging_metrics          | ModuleList         | 0      | train
2  | logging_metrics.0        | SMAPE              | 0      | train
3  | logging_metrics.1        | MAE                | 0      | train
4  | logging_metrics.2        | RMSE               | 0      | train
5  | logging_metrics.3        | MAPE               | 0      | train
6  | logging_metri

# Weather

## efficient KAN

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/Weather.py # 0.000001

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 12.0 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
12.0 K    Trainable params
0         Non-trainable params
12.0 K    Total params
0.048     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 328/328 [00:03<00:00, 103.95it/s,

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/Weather.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 30.0 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
30.0 K    Trainable params
0         Non-trainable params
30.0 K    Total params
0.120     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 328/328 [00:03<00:00, 104.68it/s,

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/eff_KAN/Weather.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | eff_KAN | 72.0 K | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
72.0 K    Trainable params
0         Non-trainable params
72.0 K    Total params
0.288     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 327/327 [00:03<00:00, 104.76it/s,

## MLP

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/Weather.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 1.3 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
1.3 K     Trainable params
0         Non-trainable params
1.3 K     Total params
0.005     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 328/328 [00:01<00:00, 165.60it/s,

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/Weather.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 3.1 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 328/328 [00:01<00:00, 165.91it/s,

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/MLP/Weather.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type    | Params | Mode 
--------------------------------------------------
0 | model         | MLP     | 8.2 K  | train
1 | loss_function | MSELoss | 0      | train
--------------------------------------------------
8.2 K     Trainable params
0         Non-trainable params
8.2 K     Total params
0.033     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 327/327 [00:02<00:00, 159.12it/s,

# Проверка формы входа выхода

In [ ]:
exp_conf = dict(
    model_name="KAN",
    dataset_name='ETTh1',

    hist_len=96,
    pred_len=48,

    max_epochs=3,

    width=[96 * 7, 50, 50 , 48 * 7],
    grid=3,
    k=3,

    lr=0.01,

    save_root="./save",

    batch_size=64,
    optimizer="AdamW",
    optimizer_betas=(0.95, 0.9),
    optimizer_weight_decay=1e-5,
    lr_scheduler='StepLR',
    lr_step_size=1,
    lr_gamma=0.5,
    gradient_clip_val=5,
    val_metric="val/loss",
    test_metric="test/mae",
    es_patience=10,

    norm_time_feature=False,
    time_feature_cls=["tod", "dow"],

    num_workers=2,

    var_num=7,
    freq=60,
    data_split=[8640, 2880, 2880],
    data_root="EasyTSF/dataset",
    devices="auto",

    include_time_feature = False,

)


In [ ]:
exp_conf = dict(
    model_name="KAN",
    dataset_name='ETTh1',

    hist_len=48,
    pred_len=12,

    max_epochs=5,

    width=[48 * 7, 20 , 12 * 7],
    grid=3,
    k=3,

    lr=0.01,

    save_root="./save",

    batch_size=64,
    optimizer="AdamW",
    optimizer_betas=(0.95, 0.9),
    optimizer_weight_decay=1e-5,
    lr_scheduler='StepLR',
    lr_step_size=1,
    lr_gamma=0.5,
    gradient_clip_val=5,
    val_metric="val/loss",
    test_metric="test/mae",
    es_patience=10,

    norm_time_feature=False,
    time_feature_cls=["tod", "dow"],

    num_workers=2,

    var_num=7,
    freq=60,
    data_split=[12194, 2613, 2613],
    devices="auto",

    include_time_feature = False,
    file_format="csv",
    data_root="drive/MyDrive/VKR/Data/Time series",
)

In [ ]:
import os

import lightning.pytorch as pl
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


class GeneralTSFDataset(Dataset):
    def __init__(self, hist_len, pred_len, variable, time_feature, model_name, include_time_feature):
        self.hist_len = hist_len
        self.pred_len = pred_len
        self.variable = variable
        self.time_feature = time_feature
        self.model_name = model_name
        self.include_time_feature = include_time_feature

    def __getitem__(self, index):
        hist_start = index
        hist_end = index + self.hist_len
        pred_end = hist_end + self.pred_len

        var_x = self.variable[hist_start:hist_end, ...]
        tf_x = self.time_feature[hist_start:hist_end, ...]

        var_y = self.variable[hist_end:pred_end, ...]
        tf_y = self.time_feature[hist_end:pred_end, ...]

        if self.include_time_feature: # static features should not be forecasted
          # concatenate all features
          var_x = np.concatenate((var_x, tf_x), axis=-1)
          var_y = np.concatenate((var_y, tf_y), axis=-1)


        if self.model_name == "KAN":
          var_x = var_x.reshape(-1)
          var_y = var_y.reshape(-1)

        return var_x, var_y

    def __len__(self):
        return len(self.variable) - (self.hist_len + self.pred_len) + 1


class DataInterface(pl.LightningDataModule):

    def __init__(self, **kwargs):
        super().__init__()
        self.num_workers = kwargs['num_workers']
        self.batch_size = kwargs['batch_size']
        self.hist_len = kwargs['hist_len']
        self.pred_len = kwargs['pred_len']
        self.norm_time_feature = kwargs['norm_time_feature']
        self.train_len, self.val_len, self.test_len = kwargs['data_split']
        self.time_feature_cls = kwargs['time_feature_cls']

        self.data_path = os.path.join(kwargs['data_root'], "{}.npz".format(kwargs['dataset_name']))
        self.config = kwargs

        self.variable, self.time_feature = self.__read_data__()

    def __read_data__(self):
        data = np.load(self.data_path)
        variable = data['variable']
        timestamp = pd.DatetimeIndex(data['timestamp'])

        # time_feature
        time_feature = []
        for tf_cls in self.time_feature_cls:
            if tf_cls == "tod":
                tod_size = int((24 * 60) / self.config['freq']) - 1
                tod = np.array(list(map(lambda x: ((60 * x.hour + x.minute) / self.config['freq']), timestamp)))
                if self.norm_time_feature:
                    time_feature.append(tod / tod_size)
                else:
                    time_feature.append(tod)
            elif tf_cls == "dow":
                dow_size = 7 - 1
                dow = np.array(timestamp.dayofweek)  # 0 ~ 6
                if self.norm_time_feature:
                    time_feature.append(dow / dow_size)
                else:
                    time_feature.append(dow)
            elif tf_cls == "dom":
                dom_size = 31 - 1
                dom = np.array(timestamp.day) - 1  # 0 ~ 30
                if self.norm_time_feature:
                    time_feature.append(dom / dom_size)
                else:
                    time_feature.append(dom)
            elif tf_cls == "doy":
                doy_size = 366 - 1
                doy = np.array(timestamp.dayofyear) - 1  # 0 ~ 181
                if self.norm_time_feature:
                    time_feature.append(doy / doy_size)
                else:
                    time_feature.append(doy)
            else:
                raise NotImplementedError

        return variable, np.stack(time_feature, axis=-1)


    def train_dataloader(self):
        return DataLoader(
            dataset=GeneralTSFDataset(
                self.hist_len,
                self.pred_len,
                self.variable[:self.train_len].copy(),
                self.time_feature[:self.train_len].copy(),
                self.config["model_name"],
                self.config['include_time_feature'],
            ),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            drop_last=True
        )

    def val_dataloader(self):
        return DataLoader(
            dataset=GeneralTSFDataset(
                self.hist_len,
                self.pred_len,
                self.variable[self.train_len - self.hist_len:self.train_len + self.val_len].copy(),
                self.time_feature[self.train_len - self.hist_len:self.train_len + self.val_len].copy(),
                self.config["model_name"],
                self.config['include_time_feature'],
            ),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            drop_last=False
        )

    def test_dataloader(self):
        return DataLoader(
            dataset=GeneralTSFDataset(
                self.hist_len,
                self.pred_len,
                self.variable[self.train_len + self.val_len - self.hist_len:].copy(),
                self.time_feature[self.train_len + self.val_len - self.hist_len:].copy(),
                self.config["model_name"],
                self.config['include_time_feature'],
            ),
            batch_size=1,
            num_workers=self.num_workers,
            shuffle=False
        )


ModuleNotFoundError: No module named 'lightning'

In [ ]:
from EasyTSF.easytsf.runner.data_runner import DataInterface

In [ ]:
data_module = DataInterface(**exp_conf)
print(data_module.time_feature.shape)

(17420, 2)


In [ ]:
train_loader = data_module.train_dataloader()
len(train_loader)

189

In [ ]:
for i, batch in enumerate(train_loader):
  if i == 0:
    var_x, var_y = [_.float() for _ in batch]
    print(var_x.shape, var_y.shape)
    # print(var_x[0], marker_x[0], var_y[0], marker_y[0])

torch.Size([64, 336]) torch.Size([64, 84])


In [ ]:
var_x[0].shape

torch.Size([336])

In [ ]:
var_y[:, -48:].shape

torch.Size([64, 48])

In [ ]:
import importlib
import inspect
import os

import lightning.pytorch as L
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lrs

from kan import KAN


class LTSFRunner(L.LightningModule):
    def __init__(self, **kargs):
        super().__init__()
        self.save_hyperparameters()
        self.load_model()
        self.configure_loss()

        stat = np.load(os.path.join(self.hparams.data_root, '{}.npz'.format(self.hparams.dataset_name)))
        self.register_buffer('mean', torch.tensor(stat['mean']).float())
        self.register_buffer('std', torch.tensor(stat['std']).float())

    def forward(self, batch, batch_idx):
        var_x, var_y = [_.float() for _ in batch]
        if self.hparams.model_name == "KAN" or self.hparams.model_name == "MLP":
          label = var_y[:, -self.hparams.pred_len:]
          prediction = self.model(var_x)[:, -self.hparams.pred_len:]
        else:
          label = var_y[:, -self.hparams.pred_len:, :]
          prediction = self.model(var_x)[:, -self.hparams.pred_len:, :]
        return prediction, label

    def training_step(self, batch, batch_idx):
        loss = self.loss_function(*self.forward(batch, batch_idx))
        self.log('train/loss', loss, on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.loss_function(*self.forward(batch, batch_idx))
        self.log('val/loss', loss, on_step=False, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss

    def test_step(self, batch, batch_idx):
        prediction, label = self.forward(batch, batch_idx)
        mae = torch.nn.functional.l1_loss(prediction, label)
        mse = torch.nn.functional.mse_loss(prediction, label)
        self.log('test/mae', mae, on_step=False, on_epoch=True, sync_dist=True)
        self.log('test/mse', mse, on_step=False, on_epoch=True, sync_dist=True)

    def configure_loss(self):
        self.loss_function = nn.MSELoss()

    def configure_optimizers(self):
        if self.hparams.optimizer == 'Adam':
            optimizer = torch.optim.Adam(
                self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.optimizer_weight_decay)
        elif self.hparams.optimizer == 'AdamW':
            optimizer = torch.optim.AdamW(
                self.parameters(), lr=self.hparams.lr, betas=(0.9, 0.95), weight_decay=1e-5)
        else:
            raise ValueError('Invalid optimizer type!')

        if self.hparams.lr_scheduler == 'StepLR':
            lr_scheduler = {
                "scheduler": lrs.StepLR(
                    optimizer, step_size=self.hparams.lr_step_size, gamma=self.hparams.lr_gamma)
            }
        elif self.hparams.lr_scheduler == 'MultiStepLR':
            lr_scheduler = {
                "scheduler": lrs.MultiStepLR(
                    optimizer, milestones=self.hparams.milestones, gamma=self.hparams.gamma)
            }
        elif self.hparams.lr_scheduler == 'ReduceLROnPlateau':
            lr_scheduler = {
                "scheduler": lrs.ReduceLROnPlateau(
                    optimizer, mode='min', factor=self.hparams.lrs_factor, patience=self.hparams.lrs_patience),
                "monitor": self.hparams.val_metric
            }
        elif self.hparams.lr_scheduler == 'WSD':
            assert self.hparams.lr_warmup_end_epochs < self.hparams.lr_stable_end_epochs < self.hparams.max_epochs

            def wsd_lr_lambda(epoch):
                if epoch < self.hparams.lr_warmup_end_epochs:
                    return (epoch + 1) / self.hparams.lr_warmup_end_epochs
                if self.hparams.lr_warmup_end_epochs <= epoch < self.hparams.lr_stable_end_epochs:
                    return 1.0
                if self.hparams.lr_stable_end_epochs <= epoch <= self.hparams.max_epochs:
                    return (epoch + 1 - self.hparams.lr_stable_end_epochs) / (
                            self.hparams.max_epochs - self.hparams.lr_stable_end_epochs)

            lr_scheduler = {
                "scheduler": lrs.LambdaLR(optimizer, lr_lambda=wsd_lr_lambda),
            }
        else:
            raise ValueError('Invalid lr_scheduler type!')

        return {
            "optimizer": optimizer,
            "lr_scheduler": lr_scheduler,
        }

    def load_model(self):
        model_name = self.hparams.model_name
        if model_name == "KAN":
          self.model = self.instancialize(KAN)
        else:
          Model = getattr(importlib.import_module('.' + model_name, package='easytsf.model'), model_name)
          self.model = self.instancialize(Model)

    def instancialize(self, Model):
        """ Instancialize a model using the corresponding parameters
            from self.hparams dictionary. You can also input any args
            to overwrite the corresponding value in self.hparams.
        """
        model_class_args = inspect.getfullargspec(Model.__init__).args[1:]  # 获取模型参数
        interface_args = self.hparams.keys()
        model_args_instance = {}
        for arg in model_class_args:
            if arg in interface_args:
                model_args_instance[arg] = getattr(self.hparams, arg)
        return Model(**model_args_instance)

    def inverse_transform_var(self, data):
        return (data * self.std) + self.mean

    def inverse_transform_time_marker(self, time_marker):
        time_marker[..., 0] = time_marker[..., 0] * (int((24 * 60) / self.hparams.freq - 1))
        time_marker[..., 1] = time_marker[..., 1] * 6
        time_marker[..., 2] = time_marker[..., 2] * 30
        time_marker[..., 3] = time_marker[..., 3] * 365

        if "max_event_per_day" in self.hparams:
            time_marker[..., -1] = time_marker[..., -1] * self.hparams.max_event_per_day

        return time_marker


from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint, EarlyStopping


callbacks = [
        ModelCheckpoint(
            monitor=exp_conf["val_metric"],
            mode="min",
            save_top_k=1,
            save_last=False,
            every_n_epochs=1,
        ),
        EarlyStopping(
            monitor=exp_conf["val_metric"],
            mode='min',
            patience=exp_conf["es_patience"],
        ),
        LearningRateMonitor(logging_interval="epoch")
    ]

trainer = L.Trainer(
        devices=exp_conf["devices"],
        precision=exp_conf["precision"] if "precision" in exp_conf else "32-true",
        callbacks=callbacks,
        max_epochs=exp_conf["max_epochs"],
        gradient_clip_algorithm=exp_conf["gradient_clip_algorithm"] if "gradient_clip_algorithm" in exp_conf else "norm",
        gradient_clip_val=exp_conf["gradient_clip_val"],
        default_root_dir=exp_conf["save_root"],
        deterministic=True,
    )

model = LTSFRunner(**exp_conf)

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


ModuleNotFoundError: No module named 'easytsf'

In [ ]:
from EasyTSF.easytsf.runner.exp_runner import LTSFRunner

In [ ]:
model = LTSFRunner(**exp_conf)

ModuleNotFoundError: No module named 'easytsf'

In [ ]:
# trainer.fit(model=model, datamodule=data_module)
model.model(var_x)[:, -48:]

tensor([[ 0.0474, -0.0729,  0.0451,  ..., -0.0263, -0.0295,  0.0290],
        [ 0.0439, -0.0752,  0.0250,  ..., -0.0383, -0.0275,  0.0007],
        [ 0.0475, -0.0649,  0.0261,  ..., -0.0118, -0.0014,  0.0234],
        ...,
        [ 0.0388, -0.0703,  0.0727,  ..., -0.0029, -0.0036,  0.0415],
        [ 0.0450, -0.0874,  0.0822,  ...,  0.0106, -0.0190,  0.0359],
        [ 0.0598, -0.0825,  0.0725,  ..., -0.0047, -0.0115,  0.0508]],
       grad_fn=<SliceBackward0>)

# RMoK

In [ ]:
!python EasyTSF/train.py -c EasyTSF/config/reproduce_conf/RMoK/ETTh1_96for96.py

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type      | Params | Mode 
----------------------------------------------------
0 | model         | DenseRMoK | 130 K  | train
1 | loss_function | MSELoss   | 0      | train
----------------------------------------------------
130 K     Trainable params
0         Non-trainable params
130 K     Total params
0.520     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode
Epoch 0: 100% 132/132 [00:02<00:00, 5